[View in Colaboratory](https://colab.research.google.com/github/shalundia/Kaggle/blob/master/digit-recognizer/CNN.ipynb)

In [1]:
!git clone "https://github.com/shalundia/Kaggle"

Cloning into 'Kaggle'...
remote: Counting objects: 59, done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 59 (delta 23), reused 43 (delta 14), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [2]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
data=np.loadtxt(open('Kaggle/digit-recognizer/train.csv', "rb"), delimiter=",", skiprows=1)
x_train = data[:,1:]
y_train = data[:,0]

test=np.loadtxt(open('Kaggle/digit-recognizer/test.csv', "rb"), delimiter=",", skiprows=1)
x_test = test

print(x_train.shape[0])
print(x_test.shape[0])

Using TensorFlow backend.


42000
28000


In [3]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)
          #validation_data=(x_test, y_test))
#score = model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

x_train shape: (42000, 28, 28, 1)
42000 train samples
28000 test samples
Epoch 1/12
42000/42000 [==============================] - 113s 3ms/step - loss: 0.3180 - acc: 0.9012
Epoch 2/12
42000/42000 [==============================] - 113s 3ms/step - loss: 0.1061 - acc: 0.9679
Epoch 3/12
42000/42000 [==============================] - 113s 3ms/step - loss: 0.0766 - acc: 0.9769
Epoch 4/12
42000/42000 [==============================] - 113s 3ms/step - loss: 0.0612 - acc: 0.9820
Epoch 5/12
42000/42000 [==============================] - 113s 3ms/step - loss: 0.0509 - acc: 0.9846
Epoch 6/12
42000/42000 [==============================] - 114s 3ms/step - loss: 0.0458 - acc: 0.9860
Epoch 7/12
42000/42000 [==============================] - 113s 3ms/step - loss: 0.0415 - acc: 0.9876
Epoch 8/12
42000/42000 [==============================] - 113s 3ms/step - loss: 0.0353 - acc: 0.9899
Epoch 9/12
42000/42000 [==============================] - 113s 3ms/step - loss: 0.0334 - acc: 0.9897
Epoch 10/12
42000/

In [0]:
prediction=model.predict(x_test,batch_size=batch_size)

submission=np.argmax(prediction,axis=1)

import pandas as pd
sub=pd.DataFrame()
sub['ImageId']=np.arange(start=1,stop=(len(test)+1))
sub['Label']=submission
sub.to_csv("submission.csv",index=False)


from google.colab import files
files.download('submission.csv')

In [8]:
prediction[0:3,:]

array([[4.65541472e-09, 1.34197653e-10, 9.99999166e-01, 7.93807601e-07,
        7.27074928e-11, 6.17560933e-11, 8.38700567e-11, 9.65903002e-09,
        7.92610866e-09, 1.47224052e-10],
       [9.99911904e-01, 1.20163692e-08, 3.04700166e-06, 1.24623091e-06,
        1.60426950e-09, 6.64504332e-05, 1.29396167e-05, 4.31762328e-06,
        9.87512383e-09, 1.03835099e-07],
       [2.27875219e-11, 3.36286068e-11, 1.83437336e-08, 5.15478362e-08,
        1.05014522e-06, 1.09607385e-07, 2.09031403e-12, 3.01166779e-07,
        4.60854537e-08, 9.99998450e-01]], dtype=float32)

In [0]:
submission=np.argmax(prediction,axis=1)


In [10]:
submission[0:5]

array([2, 0, 9, 9, 3])